<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [35]:
from sklearn.datasets import load_boston
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [2]:
RANDOM_STATE = 42

In [3]:
dataset = load_boston()
X = pd.DataFrame(dataset.data)
X.columns = dataset.feature_names
y = dataset.target

1. Разделите выборку на обучающую и тестовую в отношении 80%/20%

In [4]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

2. Обучите стандартную регрессию, а также Ridge и  Lasso и параметрами по умолчанию и выведите их R2 на тестовой выборке

In [5]:
from sklearn.linear_model import LinearRegression,Ridge,Lasso
from sklearn.metrics import r2_score

lr = LinearRegression()
lr.fit(X_train, y_train)
y_pred_lr = lr.predict(X_test)
r2_lr = r2_score(y_test, y_pred_lr)

rg = Ridge()
rg.fit(X_train, y_train)
y_pred_rg = rg.predict(X_test)
r2_rg = r2_score(y_test, y_pred_rg)

ls = Lasso()
ls.fit(X_train, y_train)
y_pred_ls = ls.predict(X_test)
r2_ls = r2_score(y_test, y_pred_ls)

print(f"R2 (LinearRegression): {r2_lr:.3f}")
print(f"R2 (Ridge): {r2_rg:.3f}")
print(f"R2 (Lasso): {r2_ls:.3f}")

R2 (LinearRegression): 0.669
R2 (Ridge): 0.666
R2 (Lasso): 0.667


3. Для Ridge и Lasso подберите коэффициент регуляризации(используйте GridSearchCV, RidgeCV, LassoCV) в пределах от $10^{-5}$ до $10^5$ (по степеням 10). Посчитайте R2 на тестовой выборке по лучшим моделям и сравните с предыдущими результатами. Напишите как изменился результат

In [6]:
from sklearn.model_selection import GridSearchCV
alphas = 10**np.linspace(-5,5,11)
grid_search_cv_rg = GridSearchCV(Ridge(), [{"alpha": alphas}], scoring='r2', cv=5).fit(X_train, y_train)
best_rg = grid_search_cv_rg.best_estimator_
y_pred_rg = best_rg.predict(X_test)
r2_rg = r2_score(y_test, y_pred_rg)

grid_search_cv_ls = GridSearchCV(Lasso(), [{"alpha": alphas}], scoring='r2', cv=5).fit(X_train, y_train)
best_ls = grid_search_cv_ls.best_estimator_
y_pred_ls = best_ls.predict(X_test)
r2_ls = r2_score(y_test, y_pred_ls)

print('GridSearchCV')
print(f"R2 (Ridge): {r2_rg:.3f}")
print(f"R2 (Lasso): {r2_ls:.3f}")

GridSearchCV
R2 (Ridge): 0.669
R2 (Lasso): 0.669


In [7]:
from sklearn.linear_model import RidgeCV, LassoCV
ridge_clf = RidgeCV(alphas=alphas,scoring='r2').fit(X_train, y_train)
y_pred_rg = ridge_clf.predict(X_test)
r2_rg = r2_score(y_test, y_pred_rg)

lasso_clf = LassoCV(alphas=alphas).fit(X_train, y_train)
y_pred_ls = lasso_clf.predict(X_test)
r2_ls = r2_score(y_test, y_pred_ls)

print(f"R2 (RidgeCV): {r2_rg:.3f}")
print(f"R2 (LassoCV): {r2_ls:.3f}")

R2 (RidgeCV): 0.669
R2 (LassoCV): 0.669


In [8]:
#Результат изменился не значительно

4. Проведите масштабирование выборки(используйте Pipeline, StandardScaler, MinMaxScaler), посчитайте R2 и сравните с предыдущими результатами. Напишите как изменился результат

In [9]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

pipeline_SS = Pipeline([
    ('scaling', StandardScaler()),
    ('regression', Ridge())
])

pipeline_MMS = Pipeline([
    ('scaling', MinMaxScaler()),
    ('regression', Ridge())
])

model1 = pipeline_SS.fit(X_train, y_train)

model2 = pipeline_MMS.fit(X_train, y_train)

print('Ridge')
print(f"R2 (StandardScaler): {model1.score(X_test,y_test):.3f}")
print(f"R2 (MinMaxScaler): {model2.score(X_test,y_test):.3f}")

#Результат не много улучшился по MinMaxScale

Ridge
R2 (StandardScaler): 0.668
R2 (MinMaxScaler): 0.676


In [10]:
pipeline_SS = Pipeline([
    ('scaling', StandardScaler()),
    ('regression', Lasso())
])

pipeline_MMS = Pipeline([
    ('scaling', MinMaxScaler()),
    ('regression', Lasso())
])

model1 = pipeline_SS.fit(X_train, y_train)

model2 = pipeline_MMS.fit(X_train, y_train)

print('Lasso')
print(f"R2 (StandardScaler): {model1.score(X_test,y_test):.3f}")
print(f"R2 (MinMaxScaler): {model2.score(X_test,y_test):.3f}")

#Значительно ухудшился результат Lasso MinMaxScaler

Lasso
R2 (StandardScaler): 0.624
R2 (MinMaxScaler): 0.257


5. Подберите коэффициент регуляризации для Ridge и Lasso на масштабированных данных, посчитайте R2 и сравните с предыдущими результатами. Напишите как изменился результат

In [11]:
pipe = Pipeline(steps=[
    ('scaler', 'passthrough'),
    ('clf', 'passthrough')
])

params = [{
    'scaler': [StandardScaler(), MinMaxScaler()],
    'clf': [Ridge()],
    'clf__alpha': alphas
}]
search = GridSearchCV(pipe, params, scoring='r2', cv=10, n_jobs=-1).fit(X_train, y_train)
best_alpha_rid = search.best_params_["clf__alpha"]

print("Best Ridge alpha = %.5f" % best_alpha_rid)
print(f"R2 : {search.score(X_test,y_test):.3f}")

Best Ridge alpha = 1.00000
R2 : 0.676


In [12]:
pipe = Pipeline(steps=[
    ('scaler', 'passthrough'),
    ('clf', 'passthrough')
])

params = [{
    'scaler': [StandardScaler(), MinMaxScaler()],
    'clf': [Lasso()],
    'clf__alpha': alphas
}]
search = GridSearchCV(pipe, params, scoring='r2', cv=10, n_jobs=-1).fit(X_train, y_train)
best_alpha_ls = search.best_params_["clf__alpha"]

print("Best Lasso alpha = %.5f" % best_alpha_ls)
print(f"R2 : {search.score(X_test,y_test):.3f}")

#Рез-тат Lasso улучшился 

Best Lasso alpha = 0.01000
R2 : 0.668


6. Добавьте попарные произведения признаков и их квадраты (используйте PolynomialFeatures) на масштабированных признаках, посчитайте R2 и сравните с предыдущими результатами. Напишите как изменился результат

In [13]:
from sklearn.preprocessing import PolynomialFeatures

poly = PolynomialFeatures(degree=2, interaction_only=True)

pipeline_SS = Pipeline([
    ('scaling', StandardScaler()),
    ('poly', poly),
    ('regression', Ridge())
])

pipeline_MMS = Pipeline([
    ('scaling', MinMaxScaler()),
    ('poly', poly),
    ('regression', Ridge())

])


model1 = pipeline_SS.fit(X_train, y_train)

model2 = pipeline_MMS.fit(X_train, y_train)

print('Ridge')
print(f"R2 (StandardScaler): {model1.score(X_test,y_test):.3f}")
print(f"R2 (MinMaxScaler): {model2.score(X_test,y_test):.3f}")
#Результат значительно улучшился

Ridge
R2 (StandardScaler): 0.848
R2 (MinMaxScaler): 0.803


In [14]:
pipeline_SS = Pipeline([
    ('scaling', StandardScaler()),
    ('poly', poly),
    ('regression', Lasso())
])

pipeline_MMS = Pipeline([
    ('scaling', MinMaxScaler()),
    ('poly', poly),
    ('regression', Lasso())

])


model1 = pipeline_SS.fit(X_train, y_train)

model2 = pipeline_MMS.fit(X_train, y_train)

print('Lasso')
print(f"R2 (StandardScaler): {model1.score(X_test,y_test):.3f}")
print(f"R2 (MinMaxScaler): {model2.score(X_test,y_test):.3f}")
#Результат значительно улучшился

Lasso
R2 (StandardScaler): 0.736
R2 (MinMaxScaler): 0.269


7. Подберите наилучшую модель (используйте Pipeline, GridSearchSCV) подбирая тип регуляризации (L1,L2), коэффициент регуляризации, метод масштабирования и степень полинома в PolynomialFeatures. Выведите итоговые параметры и результат R2. Напишите как изменился R2 по сравнению с предыдущими экспериментами

In [15]:
%%time

pipe = Pipeline(steps=[
    ('scaler', 'passthrough'),
    ('poly', 'passthrough'),
    ('clf', 'passthrough')
])

params = [{
    'scaler': [None, StandardScaler(), MinMaxScaler()],
    'poly': [None, PolynomialFeatures(degree=2,interaction_only=True),PolynomialFeatures(degree=3,interaction_only=True),PolynomialFeatures(degree=2,interaction_only=False),PolynomialFeatures(degree=3,interaction_only=False)],
    'clf': [Ridge(), Lasso()],
    'clf__alpha': alphas
}]

search = GridSearchCV(pipe, params, scoring='r2', cv=10, n_jobs=-1).fit(X_train, y_train)


print(f'best model - {search.best_params_["clf"]}\n'\
      f'score - {search.score(X_test, y_test)}\n'\
      f'scaler - {search.best_params_["scaler"]}\n'\
      f'poly - {search.best_params_["poly"]}')

#Лучший результат

best model - Ridge(alpha=0.01)
score - 0.8682246607667993
scaler - MinMaxScaler()
poly - PolynomialFeatures(degree=3, interaction_only=True)
Wall time: 1min 55s


http://archive.ics.uci.edu/ml/datasets/Adult

In [53]:
link = 'https://raw.githubusercontent.com/jbrownlee/Datasets/master/adult-all.csv'
data = pd.read_csv(link, header=None)

In [54]:
data.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


8. Разделите выборку на признаки и целевую переменную(колонка со зачениями {<=50K,>50K}). Замените целевую переменную на числовые значения.

In [55]:
df = data.copy()
df.iloc[:,14] = df.iloc[:,14].replace(['<=50K','>50K'],[0,1])

In [56]:
X = df.iloc[:,:-1]
y = df.iloc[:,-1]

9. Выясните, присутствуют ли в данных пропуски. Заполните их самыми частыми значениями (испольуйте SimpleImputer)

In [20]:
df.isnull().sum()
#Пропусков нет

0     0
1     0
2     0
3     0
4     0
5     0
6     0
7     0
8     0
9     0
10    0
11    0
12    0
13    0
14    0
dtype: int64

10. Выберите колонки с числовыми и категориальными переменными.

In [22]:
cols = X.columns

X_num_cols = X._get_numeric_data().columns
X_cat_cols = list(set(cols) - set(X_num_cols))

In [23]:
X_num_cols = X_num_cols.to_list()

In [24]:
X_cat_cols

[1, 3, 5, 6, 7, 8, 9, 13]

In [25]:
X_num_cols

[0, 2, 4, 10, 11, 12]

11. Создайте пайплайн по обработке колонок(используйте OneHotEncoder,MinMaxScaler).

In [26]:
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

In [39]:
categorical_transformer = OneHotEncoder(handle_unknown="ignore")

numeric_transformer = MinMaxScaler()

preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, X_num_cols),
        ("cat", categorical_transformer, X_cat_cols),
    ]
)

clf = Pipeline(
    steps=[("preprocessor", preprocessor), ("classifier", LogisticRegression())]
)

12. Посчитайте метрики accuracy и f1_score на предсказании только самого частого класса в целевой переменной.

In [84]:
y.value_counts()

0    34014
1    11208
Name: 14, dtype: int64

In [90]:
print('Constant prediction')
print(f'Accuracy: {accuracy_score(y, np.array([0 for _ in range(len(y))]))}')
print(f'F1-score: {f1_score(y, np.array([0 for _ in range(len(y))]))}')

Constant prediction
Accuracy: 0.7521560302507629
F1-score: 0.0


13. Посчитайте cross_val_score по алгоритмам LogisticRegression, SVC, LinearSVC по метрикам accuracy и f1_score.
Напишите удалось ли превзойти предыдущий результат.

In [45]:
from sklearn.model_selection import cross_val_score
from sklearn.svm import SVC, LinearSVC

clf_lg = Pipeline(
    steps=[("preprocessor", preprocessor), ("classifier", LogisticRegression())]
)

print('LogisticRegression')
print('accuracy', cross_val_score(clf_lg, X, y, cv=5, scoring = 'accuracy').mean())
print('f1', cross_val_score(clf_lg, X, y, cv=5, scoring = 'f1').mean())



print()
print('-----------------------------------------')
print()

clf_lsvc = Pipeline(
    steps=[("preprocessor", preprocessor), ("classifier", LinearSVC())]
)
print('LinearSVC')
print('accuracy', cross_val_score(clf_lsvc, X, y, cv=5, scoring = 'accuracy').mean())
print('f1', cross_val_score(clf_lsvc, X, y, cv=5, scoring = 'f1').mean())

print()
print('-----------------------------------------')
print()

clf_svc = Pipeline(
    steps=[("preprocessor", preprocessor), ("classifier", SVC())]
)
print('SVC')
print('accuracy', cross_val_score(clf_svc, X, y, cv=5, scoring = 'accuracy').mean())
print('f1', cross_val_score(clf_svc, X, y, cv=5, scoring = 'f1').mean())


LogisticRegression
accuracy 0.8511323258508231
f1 0.6564158702328787

-----------------------------------------

LinearSVC
accuracy 0.8529135478362626
f1 0.6578163598233313

-----------------------------------------

SVC
accuracy 0.8399739469040318
f1 0.6200828429738441


14. Можно заметить что в данных присутствуют значения '?', замените их самыми частыми значениями (испольуйте SimpleImputer)

In [28]:
(X == '?').sum()

0        0
1     2799
2        0
3        0
4        0
5        0
6     2809
7        0
8        0
9        0
10       0
11       0
12       0
13     857
dtype: int64

In [29]:
from sklearn.impute import SimpleImputer
data_columns = X.columns
imputer = SimpleImputer(missing_values='?', strategy='most_frequent')
new_data = pd.DataFrame(imputer.fit_transform(X), columns=data_columns)
(new_data== '?').sum()

0     0
1     0
2     0
3     0
4     0
5     0
6     0
7     0
8     0
9     0
10    0
11    0
12    0
13    0
dtype: int64

15. Посчитайте cross_val_score на новых данных. Напишите удалось ли улучшить результат.

In [46]:
clf_lg = Pipeline(
    steps=[("preprocessor", preprocessor), ("classifier", LogisticRegression())]
)

print('LogisticRegression')
print('accuracy', cross_val_score(clf_lg, new_data, y, cv=5, scoring = 'accuracy').mean())
print('f1', cross_val_score(clf_lg, new_data, y, cv=5, scoring = 'f1').mean())



print()
print('-----------------------------------------')
print()

clf_lsvc = Pipeline(
    steps=[("preprocessor", preprocessor), ("classifier", LinearSVC())]
)
print('LinearSVC')
print('accuracy', cross_val_score(clf_lsvc, new_data, y, cv=5, scoring = 'accuracy').mean())
print('f1', cross_val_score(clf_lsvc, new_data, y, cv=5, scoring = 'f1').mean())

print()
print('-----------------------------------------')
print()

clf_svc = Pipeline(
    steps=[("preprocessor", preprocessor), ("classifier", SVC())]
)
print('SVC')
print('accuracy', cross_val_score(clf_svc, new_data, y, cv=5, scoring = 'accuracy').mean())
print('f1', cross_val_score(clf_svc, new_data, y, cv=5, scoring = 'f1').mean())

# Результат стал незначительно хуже на тысячные доли

LogisticRegression
accuracy 0.8505794751469846
f1 0.6539287793928741

-----------------------------------------

LinearSVC
accuracy 0.8508251523375897
f1 0.650945261381221

-----------------------------------------

SVC
accuracy 0.8395848963639935
f1 0.6169178164345149


16. Посчитайте cross_val_score, если просто удалить значения '?'. Напишите как изменился результат

In [57]:
df_clear = df.replace('?', np.nan).dropna()

In [58]:
X = df_clear.iloc[:,:-1]
y = df_clear.iloc[:,-1]

clf_lg = Pipeline(
    steps=[("preprocessor", preprocessor), ("classifier", LogisticRegression())]
)

print('LogisticRegression')
print('accuracy', cross_val_score(clf_lg, X, y, cv=5, scoring = 'accuracy').mean())
print('f1', cross_val_score(clf_lg, X, y, cv=5, scoring = 'f1').mean())



print()
print('-----------------------------------------')
print()

clf_lsvc = Pipeline(
    steps=[("preprocessor", preprocessor), ("classifier", LinearSVC())]
)
print('LinearSVC')
print('accuracy', cross_val_score(clf_lsvc, X, y, cv=5, scoring = 'accuracy').mean())
print('f1', cross_val_score(clf_lsvc, X, y, cv=5, scoring = 'f1').mean())

print()
print('-----------------------------------------')
print()

clf_svc = Pipeline(
    steps=[("preprocessor", preprocessor), ("classifier", SVC())]
)
print('SVC')
print('accuracy', cross_val_score(clf_svc, X, y, cv=5, scoring = 'accuracy').mean())
print('f1', cross_val_score(clf_svc, X, y, cv=5, scoring = 'f1').mean())

#Результат ухудшился на сотые доли

LogisticRegression
accuracy 0.8468445746108516
f1 0.6601244618889603

-----------------------------------------

LinearSVC
accuracy 0.8485030154158197
f1 0.6616132960278641

-----------------------------------------

SVC
accuracy 0.8356995943179577
f1 0.6267120053492474


 17. Посчитайте cross_val_score для RandomForestClassifier,GradientBoostingClassifier. Напишите как изменился результат и какой вывод можно из этого сделать.

In [60]:
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

clf_svc = Pipeline(
    steps=[("preprocessor", preprocessor), ("classifier", GradientBoostingClassifier())]
)
print('GradientBoostingClassifier')
print('accuracy', cross_val_score(clf_svc, X, y, cv=5, scoring = 'accuracy').mean())
print('f1', cross_val_score(clf_svc, X, y, cv=5, scoring = 'f1').mean())

print()
print('-----------------------------------------')
print()

clf_svc = Pipeline(
    steps=[("preprocessor", preprocessor), ("classifier", RandomForestClassifier())]
)
print('RandomForestClassifier')
print('accuracy', cross_val_score(clf_svc, X, y, cv=5, scoring = 'accuracy').mean())
print('f1', cross_val_score(clf_svc, X, y, cv=5, scoring = 'f1').mean())

# Результат GradientBoostingClassifier превзошел все алгоритмы, но работает дольше.

GradientBoostingClassifier
accuracy 0.8629208373582479
f1 0.6869948630650294

-----------------------------------------

RandomForestClassifier
accuracy 0.8486356413910594
f1 0.6704805561583802


18. Подберите наилучшую модель, подбирая методы обработки колонок - масштабирование признаков, кодирование признаков и заполнение пропусков. Параметры алгоритмов оставьте по умолчанию. Выведите итоговые параметры и результат accuracy и f1_score.

In [65]:
ct_1 = ColumnTransformer([
    ('scale',  MinMaxScaler(), X_num_cols),
    ('onehot', OneHotEncoder(),  X_cat_cols)
])

ct_2 = ColumnTransformer([
    ('scale',  StandardScaler(), X_num_cols),
    ('onehot', OneHotEncoder(),  X_cat_cols)
])

pipe = Pipeline(steps=[
    ('fill_missing', SimpleImputer(missing_values='?', strategy='most_frequent')),
    ('prepare_columns', 'passthrough'),
    ('clf', 'passthrough')]
)

params = [
    {
        'prepare_columns': [ct_1, ct_2],
        'clf': [
            RandomForestClassifier(),
            GradientBoostingClassifier(),
            LogisticRegression(),
            LinearSVC()
        ]
    }
]

search = GridSearchCV(pipe, params, scoring=['accuracy', 'f1'], refit='f1', n_jobs=-1)
search.fit(X, y)
print(search.best_params_)

{'clf': RandomForestClassifier(), 'prepare_columns': ColumnTransformer(transformers=[('scale', MinMaxScaler(),
                                 [0, 2, 4, 10, 11, 12]),
                                ('onehot', OneHotEncoder(),
                                 [1, 3, 5, 6, 7, 8, 9, 13])])}


In [89]:
f1_scores = np.array([search.cv_results_[f'split{i}_test_f1'][0] for i in range(5)])
accuracy_scores = np.array([search.cv_results_[f'split{i}_test_accuracy'][0] for i in range(5)])
print(f"F1-score: {np.mean(accuracy_scores[~np.isnan(accuracy_scores)])}")
print(f"Accuracy: {np.mean(f1_scores[~np.isnan(f1_scores)])}")

F1-score: 0.8476421916414293
Accuracy: 0.668513115684649
